# 1 Import Dependencies

In [27]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import RandomizedSearchCV
import math

# 3 Data Pre-Processing

In [28]:
# Landmarks einlesen

data_normal = pd.read_csv('../../Data/samples.csv', sep=',', encoding='utf-8')
data_compensation = pd.read_csv('../../Data/KeypointsExportNormal.csv', sep=',', encoding='utf-8')
# Label setzen (Kompensation = 1, keine Kompensation = 0)
data_compensation.compensation = 1
data_normal.compensation = 0
# Normal- und Kompensation-Liste zusammenführen
data = pd.concat([data_normal, data_compensation], axis=0)
data[data.frame == 1]

,path,frame,compensation,x_0,y_0,z_0,x_1,y_1,z_1,x_2,...,z_29,x_30,y_30,z_30,x_31,y_31,z_31,x_32,y_32,z_32
0,bachelorarbeiten/nils/data/videos/samples/drin...,1,0,980.270700,321.902770,-637.872200,1001.2071,298.31467,-570.74340,1012.57666,...,-608.882200,868.95660,822.7685,-425.566700,1129.271700,765.220950,-821.67800,874.072630,790.297500,-602.045400
172,bachelorarbeiten/nils/data/videos/samples/03_t...,1,0,1020.428830,267.426820,-428.964570,1037.8142,237.42938,-372.71088,1048.06870,...,-39.149357,917.36487,1188.9327,-127.484695,1046.446200,1207.436000,-247.56013,865.767760,1265.434900,-387.392430
471,bachelorarbeiten/nils/data/videos/samples/drin...,1,0,996.641600,298.614440,-453.319150,1018.8884,274.45960,-387.38930,1031.51050,...,-308.314360,849.54450,1069.6743,529.557430,1051.905400,1076.808700,-604.22864,820.501800,1113.729500,264.633480
0,/Users/salomekoller/Library/CloudStorage/OneDr...,1,1,1.200000,0.000000,2.900605,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.117923,-1.599242,2.99548,1.282079,-1.599241,2.995479
84,/Users/salomekoller/Library/CloudStorage/OneDr...,1,1,1.192955,-0.025759,2.987175,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.031801,-1.601761,2.81883,1.195505,-1.601846,2.820488


In [29]:
# Videos gruppieren und in Liste schreiben
videos_raw = list(data.groupby(data.path))
len(videos_raw)

4

# 3.1 Pre-Processing Funktionen

In [30]:
'''
Diese Funktion bringt die rohen Videos in eine neue Form, ohne die Videos zu␣
,→schneiden.
Sie wird für den Sliding Window Ansatz benötigt.
Aus den rohen Videos werden nur noch die exrahierten Datenpunkte als Liste␣
,→zurückgegeben.
'''
def remap_raw_videos(unmapped_videos):
    remapped_videos = []
    for video in unmapped_videos:
        v = video[1].reset_index()
        mapped_vid = v.loc[:, 'x_0':]
        remapped_videos.append(mapped_vid)
    return remapped_videos


In [31]:
'''
Schneiden der Videos
Diese Funktion bringt die rohen Videos in eine neue Form und schneidet sie eine␣
,→bestimmte Anzahl Frames
vor und hinter dem vertikalen Höhepunkt der trinkenden Hand ab.
Aus den rohen Videos werden nur noch die exrahierten Datenpunkte als Liste␣
,→zurückgegeben.
Inputs:
uncutted_videos: Die ungeschnittenen rohen Videos
n_frames: Die Anzahl der Frames, welche vor und nach dem Höchstpunkt der Hand␣
,→(y_16) abgeschnitten werden sollen
'''
def cut_videos(uncutted_videos, n_frames=0):
    cut_videos = []
    for idx, video in enumerate(uncutted_videos):
        v = video[1].reset_index()
        # Position des höchsten Punktes (tiefster Wert, da tiefere Werte höhere vertikale Positionen aussagen)
        minpos = np.argmin(v.y_16)
        # Falls der Höhepunkt der Hand zu nahe am Beginn / Ende des Videos ist, ist die Trinkbewegung vermutlich nicht korrekt.
        # Das Video soll dann optimalerweise gelöscht werden.
        if((minpos < 20) or (len(v) - minpos < 20)):
            print('\033[91m' + 'Video mit dem Namen\n'
            + videos_raw[idx][0]
            + '\nund Index\n' + str(idx)
            + '\nzeigt keine korrekte Trinkbewegung. Bitte entfernen.')
        cut_vid = v.loc[minpos-n_frames:minpos+n_frames, 'x_0':] # Video x rames vor und x Frames nach Höhepunkt der Hand (y_16) abschneiden
        cut_videos.append(cut_vid)
    return cut_videos

In [32]:
'''
Videos zentrieren
Diese Funktion zentriert alle Videos, in dem von allen Landmarks die Position␣
,→des Kopfes im ersten Frame abgezogen wird.
'''
def center_data(uncentered_videos):
    centered_videos = []
    for video in uncentered_videos:
        centered_video = []
        # Position des Kopfes im ersten Frame des Videos bestimmen (x, y und z-Koordinate)
        head_start = video.loc[:,'x_0':'z_0'].values[0]
    for frame in np.array(video):
        centered_frame = []
        # Frame reshapen, sodass alle Landmarks als eine Liste zählen
        landmarks = frame.reshape((33, 3))
    for landmark in landmarks:
        centered_frame.append(landmark - head_start)
        centered_video.append(list(np.array(centered_frame).flatten()))
        centered_videos.append(centered_video)
    return centered_videos


In [33]:
'''
Relative Abstände
Diese Funktion berechnet den Abstand jedes Punktes des Skeletts zum Kopf und␣
,→gibt diese anschliessend zurück.
Shape der Rückgabe: [x Anzahl Videos, x Anzahl Frames, 33 Datenpunkte]
'''
def calc_distances(raw_videos):
    # Abstand zu Kopf
    distances = []
    for video in raw_videos:
        frame_distances = []
    for frame in np.array(video):
        points = frame.reshape((33, 3))
        point_distances = []
    for k in range(len(points)):
        # Distanz einzeln von x-, y- und z-Koordinaten
        distance = points[k]-points[0]
        # Distanz mittels Formel berechnen
        point_distances.append(math.sqrt(distance[0] ** 2 + distance[1] ** 2 + distance[2] ** 2))
        frame_distances.append(point_distances)
        distances.append(frame_distances)
    return distances


In [34]:
print(videos_raw[:5])

[('/Users/salomekoller/Library/CloudStorage/OneDrive-ZHAW/8. Semester/Bachelorarbeit/Unity VR Environment/Assets/Data/KeypointsExportNormal.csv',                                                   path  frame  compensation   
0    /Users/salomekoller/Library/CloudStorage/OneDr...      1             1  \
1    /Users/salomekoller/Library/CloudStorage/OneDr...      2             1   
2    /Users/salomekoller/Library/CloudStorage/OneDr...      3             1   
3    /Users/salomekoller/Library/CloudStorage/OneDr...      4             1   
4    /Users/salomekoller/Library/CloudStorage/OneDr...      5             1   
..                                                 ...    ...           ...   
165  /Users/salomekoller/Library/CloudStorage/OneDr...     82             1   
166  /Users/salomekoller/Library/CloudStorage/OneDr...     83             1   
167  /Users/salomekoller/Library/CloudStorage/OneDr...     84             1   
168  /Users/salomekoller/Library/CloudStorage/OneDr...     85   

In [35]:
'''
Diese Funktion gibt die Labels der Videos zurück.
1 = Compensation
0 = Natural
'''
def define_labels():
    labels = []
    for i in range(len(videos_raw)):
        labels.append(np.mean(videos_raw[i][1].compensation))
    return labels
labels = define_labels()

In [36]:
'''
Sliding Windows
Diese Funktion erstellt für alle Videos von unslided_videos Sliding Windows und␣
,→gibt sie zurück.
Ausserdem werden die Labels auf die Sliding Windows korrekt verteilt.
Shape der Rückgabe: [x Anzahl Sliding Windows, {window_size} Anzahl Frames, 99␣
,→Datenpunkte]
'''
def create_sliding_windows(unslided_videos, window_size):
    videos_slided = []
    unslided_labels = define_labels()
    slided_labels = []
    for idx, unslided_video in enumerate(unslided_videos):
        video_label = unslided_labels[idx]
    for i in range(len(unslided_video) - window_size + 1):
        videos_slided.append(unslided_video[i:i+window_size])
        slided_labels.append(video_label)
    return videos_slided, slided_labels


In [37]:
'''
In dieser Funktion werden die Daten in eine geeignete Form gebracht.
Es werden die Values der einzelnen Videos in eine Liste geschrieben und␣
,→zurückgegeben.
'''
def reshape_videos(unshaped_videos):
    reshaped_videos = []
    labels = []
    for video in unshaped_videos:
        reshaped_videos.append(video.values)
    return reshaped_videos


# 3.2 Ansatz auswählen

In [38]:
# Ansatz 1: Absolute Datenpunkte mit geschnittenen Videos
videos = reshape_videos(cut_videos(videos_raw, n_frames=38))
# Ansatz 2: Zentrierte Datenpunkte mit geschnittenen Videos
# videos = center_data(cut_videos(videos_raw, n_frames=10))
# Ansatz 3: Relative Datenpunkte (Abstände zum Kopf) mit geschnittenen Videos
# videos = calc_distances(cut_videos(videos_raw, n_frames=10))
# Ansatz 4a: Sliding Windows mit absoluten Datenpunkten
# videos, labels = create_sliding_windows(remap_raw_videos(videos_raw), 10)
# Ansatz 4b: Sliding Windows mit zentrierten Datenpunkten
# videos, labels = create_sliding_windows(center_data(remap_raw_videos(videos_raw)), 10)
# Ansatz 4b: Sliding Windows mit relativen Datenpunkten
# videos, labels = create_sliding_windows(calc_distances(remap_raw_videos(videos_raw)), 10)
np.array(videos).shape

(4, 77, 99)